In [1]:
!nvidia-smi

Mon Feb 16 07:52:04 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%%writefile add.cpp
#include<iostream>
#include<math.h>
using namespace std;
void add(int n, float *x, float *y){
  for(int i=0;i<n;i++){
    y[i]=x[i]+y[i];
  }}
  int main(void){
    int N= 1<<20;
    float *x= new float[N];
    float *y= new float[N];
    for(int i=0;i<N;i++){
      x[i]=1.0f;
      y[i]=2.0f;
    }
    add(N,x,y);
    for(int i=0;i<7;i++){
      cout <<y[i]<<" ";}
    delete[] x;
    delete[] y;

  return 0;
}

Writing add.cpp


In [3]:
!g++ add.cpp -o add
!./add


3 3 3 3 3 3 3 

In [4]:
%%writefile add.cu
#include<iostream>
using namespace std;
__global__
void add(int n, float *x, float *y){
  int index= blockIdx.x * blockDim.x +threadIdx.x;
  if(index<n){
    y[index]= x[index]+y[index];
  }
}
int main(){
  int N= 1<<20;
  float *x, *y;
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));
  for(int i=0;i<N;i++){
    x[i]= 1.0f;
    y[i]= 1.0f;
  }
  int blockSize= 256;
  int numBlocks=(N+blockSize-1)/ blockSize;
  add<<<numBlocks, blockSize>>>(N,x,y); 
  cudaDeviceSynchronize();
  for (int i=0;i<7;i++){
     cout << y[i] << " ";
    }

    cudaFree(x);
    cudaFree(y);

    return 0;
}
 

Writing add.cu


In [5]:
!nvcc add.cu -o add_cuda
!./add_cuda

2 2 2 2 2 2 2 